## Initialization

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
import os
import random
import time
import pandas
import numpy as np
import struct
from PIL import Image
from PIL import ImageFilter
import pickle
import zipfile

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers import Conv2D, MaxPooling2D, Dropout, Activation, Flatten, Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.models import load_model

In [0]:
data_dir = './'
extension = '.zip'

for item in os.listdir(data_dir):
  if item.endswith(extension):
    file_name = os.path.abspath(item)
    new_dir = file_name.replace('.zip', '')
    
    # print (file_name)
    # print (new_dir)

    zip_ref = zipfile.ZipFile(file_name)
    zip_ref.extractall(new_dir)
    zip_ref.close()
    os.remove(file_name)

## Read data

In [0]:
class ReadData:
  
  def __init__(self, path, is_train=True, char_dict=None):
    self.file_count = 0
    self.is_train = is_train
    self.iter_index = 0
    self.path = path
    self.char_dict = char_dict
    self.use_filter = True
    self.use_rotation = True
  
  # Read data

  def read_one_file(self, f):
    header_size = 10
    while True:
      header = np.fromfile(f, dtype='uint8', count=header_size)
      if not header.size: break
      sample_size = header[0] + (header[1]<<8) + (header[2]<<16) + (header[3]<<24)
      tagcode = header[5] + (header[4]<<8)
      width = header[6] + (header[7]<<8)
      height = header[8] + (header[9]<<8)
      if header_size + width*height != sample_size:
        break
      try:
        image = np.fromfile(f, dtype='uint8', count=width*height).reshape((height,width))
      except:
        print (struct.pack('>H', tagcode).decode('gb2312'))
      yield image, tagcode
  
  def read_from_dir(self, gnt_dir):
    for file_name in os.listdir(gnt_dir):
      if file_name.endswith('.gnt'):
        file_path = os.path.join(gnt_dir, file_name)
        with open(file_path, 'rb') as f:
          for image, tagcode in read_one_file(f):
            yield image, tagcode
  
  def read_one_gnt_file(self):
    for file_name in os.listdir(self.path):
      if file_name.endswith('.gnt'):
        file_path = os.path.join(self.path, file_name)
        with open(file_path, 'rb') as f:
          x = []
          y = []
          for image, tagcode in self.read_one_file(f):
            x.append(image)
            y.append(tagcode)
        yield x, y
  
  def load_next_file(self):
    for x_one, y_one in self.read_one_gnt_file():
      result_x = []
      result_y = []
      for i in range(len(x_one)):
        result = self.read_convert_image(x_one[i])
        result_x.append(result)
        result_y.append(y_one[i])
        if self.use_filter:
          filtered_x = self.apply_filter(x_one[i])
          result_x.append(filtered_x)
          result_y.append(y_one[i])
        if self.use_rotation:
          rotated_x = self.rotate(x_one[i])
          result_x.append(rotated_x)
          result_y.append(y_one[i])
      x = np.array(result_x)
      y = np.array(result_y)
      self.file_count += 1
      print ('Loaded files ', self.file_count)
      yield x, y
  
  def load_all(self):
    x = []
    y = []
    for temp_x, temp_y in self.load_next_file():
      x.extend(temp_x)
      y.extend(temp_y)
    return np.array(x), np.array(y)

  def rotate(self, image):
    im = Image.fromarray(image)
    im.rotate(random.randint(10,20))
    im = im.resize([64, 64])
    new_image = np.asarray(im)
    new_image = new_image.reshape(new_image.shape[0], new_image.shape[1], 1)
    return new_image
  
  def apply_filter(self,image):
    im = Image.fromarray(image)
    filters = [ImageFilter.BLUR, ImageFilter.CONTOUR, ImageFilter.EMBOSS]
    im.filter(random.choice(filters))
    im = im.resize([64, 64])
    new_image = np.asarray(im)
    new_image = new_image.reshape(new_image.shape[0], new_image.shape[1], 1)
    return new_image
  
  def read_convert_image(self, image):
    im = Image.fromarray(image)
    im = im.resize([64, 64])
    new_image = np.asarray(im)
    new_image = new_image.reshape(new_image.shape[0], new_image.shape[1], 1)
    return new_image

In [0]:
class GetCharList:
  
  def __init__(self):
    self.train = ReadData('./HWDB1.1trn_gnt/', is_train=True)
    self.test = ReadData('./HWDB1.1tst_gnt/', is_train=False)

  def generate_char_list(self):
    if os.path.isfile('char_list'):
      with open('char_list', 'rb') as f:
        char_list = pickle.load(f)
        print ('Char list loaded')
        return char_list
    else:
      char_list = []
      for _, tagcode in self.train.read_from_dir(gnt_dir='./HWDB1.1trn_gnt/'):
        char_list.append(tagcode)
      with open('char_list', 'wb') as f:
        pickle.dump(char_list, f)
        print ('Char list generated')
      return char_list

## Build model

In [0]:
# Initialize

number_of_classes = 3755
chars = GetCharList()
chars.test.use_rotation = False
chars.test.use_filter = False

In [0]:
with open('char_list', 'rb') as f:
  char_list = pickle.load(f)

names = ['id', 'data']
formats = ['|U16', 'int64']
dtype = dict(names=names, formats=formats)
char_array = np.array(list(char_list.items()), dtype=dtype)

lb = LabelBinarizer()
lb.fit(char_array)

In [0]:
def build_model():
  model = Sequential()

  model.add(Conv2D(128, (3, 3), input_shape=(64, 64, 1)))
  model.add(BatchNormalization(axis=-1))
  model.add(Activation('relu'))
  
  model.add(Conv2D(64, (3, 3)))
  model.add(BatchNormalization(axis=-1))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(64, (3, 3)))
  model.add(BatchNormalization(axis=-1))
  model.add(Activation('relu'))
  
  model.add(Conv2D(64, (3, 3)))
  model.add(BatchNormalization(axis=-1))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))
  model.add(Dense(3755))
  model.add(Activation('softmax'))
  
  return model

## Train model

In [0]:
def train_model(X_train,y_train):
  model = build_model()
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model.summary()
  model.fit(X_train, y_train, epochs=10)
  model.save('model1.h5')

In [0]:
start_time = time.time()
X_train, y_train = chars.train.load_all()
X_train = X_train/255
y_train = lb.transform(y_train)

In [0]:
train_model(X_train, y_train)

## Test model

In [0]:
def test_model(X_test, y_test):
  model = load_model('model.h5')
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  model_score = model.evaluate(X_test, y_test)
  print ('Testing accuracy: ', model_score[1])
  print (model.metrics)

In [0]:
X_test, y_test = chars.test.load_all()
y_test = mlb.transform(y_test)
X_test = X_test/255

In [0]:
test_model(X_test, y_test)